In [1]:
import pandas as pd
import numpy as np
from os.path import expanduser

In [2]:
# path to train and testing data
PATH_train = "../data/training.csv"
PATH_test = "../data/test.csv"

In [41]:
# load the csv into pandas data frame
df = pd.read_csv(expanduser(PATH_train))

In [42]:
# the output variables value for each sample is stored as ' ' separated string
# use apply for the colmun Image of the data frame to convert the values into
# numbers
# after conversion the entry for the Image column associated with each sample
# is a numpy vector of shape (9216 ,). Each original image is of size (96x96)
df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))

In [76]:
# Check if there is any missing value
print '{} samples from the total of {} have missing values'.format(
    df.isnull().any(axis=1).sum(),df.shape[0])
print 'Missing values appear in {} different columns'.format(
    df.isnull().any(axis=0).sum())

4909 samples from the total of 7049 have missing values
Missing values appear in 28 different columns


In [77]:
# print out variables in the work space
from types import ModuleType
variables = %who_ls
for v in variables:
    if not isinstance(eval(v), ModuleType):
        print '{:<20} {:<20}'.format(v, type(eval(v)))

ModuleType           <type 'type'>       
PATH_test            <type 'str'>        
PATH_train           <type 'str'>        
df                   <class 'pandas.core.frame.DataFrame'>
expanduser           <type 'function'>   
v                    <type 'str'>        
variables            <type 'list'>       
